In [1]:
import heapq
import matplotlib.pyplot as plt
import numpy as np
import random
from enum import Enum
from math import sqrt
from scipy.stats import norm

In [2]:
class Event:
    def __init__(self, Time: float, Type: str) -> None:
        self.Time = Time
        self.Type = Type

    def __lt__(self, other):
        """
        Allows to determine comparison rules between two Event objects
        for priority queue ordering.
        """
        return self.Time < other.Time

    def __str__(self):
        return str(self.Type) + "|" + str(self.Time)

def create_event(event_time: float, event_type: str) -> Event:
    return Event(
        Time = event_time,
        Type = event_type
    )

def next_event_time(elapsed_time: float, rate: float) -> float:
    """
    Calculates time until the next event of this type.
    """
    return elapsed_time + np.random.exponential(1/rate)

In [3]:
class Infected:
    def __init__(self, Id: int, Generation: int, InfectionTime: float, InfectedAmount: int) -> None:
        self.Id = Id
        self.Generation = Generation
        self.InfectionTime = InfectionTime
        self.InfectedAmount = InfectedAmount
    
    def heal(self, HealTime: float):
        self.HealTime = HealTime

    def __lt__(self, other):
        """
        Allows to determine comparison rules between two Event objects
        for priority queue ordering.
        """
        return self.InfectionTime < other.InfectionTime

    def __str__(self):
        string = "[" + "generation: " + str(self.Generation) + ", " \
            + "time of infection: " + str(self.InfectionTime) + ", " \
            + "infected amount: " + str(self.InfectedAmount) + "]"
        return string
    
    def __repr__(self):
        return str(self)

def create_infected(id: int, generation: int, infection_time: float) -> Infected:
    return Infected(
        Id = id,
        Generation = generation,
        InfectionTime = infection_time,
        InfectedAmount = 0
    )

In [4]:
def calculate_average_sample_ci(sample_averages: list[int]) -> tuple[float, list[float], list[float]]:
    sample_variances = []
    lower_cis = []
    cis = []
    upper_cis = []

    for i in range(len(sample_averages)):
        sample_variances.append(np.var(sample_averages[0:i]))

        ci = (1.96*sqrt(sample_variances[-1]))/sqrt(len(sample_variances))

        cis.append(ci)
        lower_cis.append(sample_averages[i] - ci)
        upper_cis.append(sample_averages[i] + ci)

    return ci, lower_cis, upper_cis

def compute_cdf(data):
    # Sorts data points in ascending order
    sorted_data = np.sort(data)

    # Generates the cumulative probability distribution for each value
    # (with intentional repetitions in the data points for plotting)
    probs = np.arange(1, len(data)+1) / len(data)
    return sorted_data, probs

In [5]:
def simulate_MM1(
    lam,
    mu,
    init_queue_size = 0,
    sim_time = 100.0,
    max_queue_size = None
):

    # Initialize queue variables
    clock = 0
    event_queue = []
    customers_in_system = 0
    customers_arrived = 0
    customers_served = 0
    last_event_time = 0

    # Initialize epidemic simulation variables
    infected_queue = [] #fila de controle de pessoas infectadas
    infected_id = 0 #identificador do infectado
    healed_queue = [] #fila de curados
    leaf_amount = 0 #quantidade de folhas da arvore
    current_gen_num = 0 #controle de geracoes
    current_gen_infected = [] #controle de gerações
    generations = [] #controle de geracoes

    # Initialize priority queue with people already waiting
    for i in range(init_queue_size):
        event = create_event(
            0,
            'arrival'
        )
        heapq.heappush(event_queue, event)

        # Update number of customers in system and arrived
        customers_in_system += 1
        customers_arrived += 1

    # Generate and insert first arrival event
    event = create_event(
        next_event_time(clock, lam),
        'arrival'
    )
    heapq.heappush(event_queue, event)

    # Main simulation loop
    while clock < sim_time:

        # Remove the event from the head of the priority queue
        event = heapq.heappop(event_queue)

        # If the event's time exceeds total simulation time, stop
        if event.Time > sim_time:
            break

        # Update the simulation clock
        clock = event.Time

        if event.Type == 'arrival':
            # debug
            # print('chegada')

            # Update the number of customers in system and arrived
            customers_in_system += 1
            customers_arrived += 1

            # Adding new infected to queue:
            #   If we are on the root (patient zero), their generation is the 1st
            if customers_in_system == 1:
              gen = 0

            #   Otherwise:
            else:
              # Their generation is 1 beyond the generation of the person that infected them 
              gen = infected_queue[0].Generation + 1
              # The infector's amount of people infected is increased
              infected_queue[0].InfectedAmount += 1

            # Inserting the new infected into the infected queue
            infected = create_infected(
               id = infected_id,
               generation = gen,
               infection_time = clock
            )
            infected_queue.append(infected)

            # Updating the next infected's id
            infected_id += 1

            # infected = [infected_id,gen,0,clock] #id, geracao, qtd_filhos, t-entrada

            # Schedule the next arrival event
            event = create_event(
                next_event_time(clock, lam),
                'arrival'
            )
            heapq.heappush(event_queue, event)

            # If this is the only customer in the system, schedule their service
            if customers_in_system == 1:
                event = create_event(
                    next_event_time(clock, mu),
                    'service'
                )
                heapq.heappush(event_queue, event)

        elif event.Type == 'service':
            #debug
            # print('serviço')

            # Update the number of customers in system and served
            customers_in_system -= 1
            customers_served += 1

            #novo
            healed = infected_queue.pop(0)
            if healed.InfectedAmount == 0:
              leaf_amount += 1
            healed_queue.append(healed)
            if current_gen_num == healed.Generation:
              current_gen_infected.append(healed.Id)
            else:
              generations.append(current_gen_infected)
              current_gen_infected = [healed.Id]
              current_gen_num += 1
            if customers_in_system <= 0:
              generations.append(current_gen_infected)
              break

            # If there are still customers in the system, schedule the next service
            if customers_in_system > 0:
                event = create_event(
                    next_event_time(clock, mu),
                    'service'
                )
                heapq.heappush(event_queue, event)

        #debug
        # print(infected_queue)  

    print(healed_queue)
    print("quantidade de folhas da arvore: " + str(leaf_amount))
    for i in range(len(generations)):
      print("geracao " + str(i) + ":" + str(generations[i]))

In [6]:
simulate_MM1(
    lam = 0.5,
    mu = 0.5
)

[[generation: 0, time of infection: 2.6144007904793356, infected amount: 1], [generation: 1, time of infection: 4.11585107421652, infected amount: 2], [generation: 2, time of infection: 5.184059309387069, infected amount: 0], [generation: 2, time of infection: 6.915752075144452, infected amount: 0]]
quantidade de folhas da arvore: 2
geracao 0:[0]
geracao 1:[1]
geracao 2:[2, 3]
